In [588]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [589]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [590]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [591]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            #deme = i.split(".")[1]
           # print(deme)
            #interval = 
            #print(interval)
#             if "\n" in i.split(".")[2]:
#                 interval = i.split(".")[2][0:2]
#             else:
            interval = i.split(".")[1]
            next_interval = int(interval)+6 #averaging over two weeks to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/6)*(np.log(input_df[i].astype("float")) - np.log(input_df["Ne"+"." + str(next_interval)].astype('float')))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [592]:
log_file_path = "../multitree_coalescent/results/multicoal_updated_la_clusters_with_metadata_10_07_gtr.log"


In [593]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#burnin
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent)
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()
Ne_df.head()

1277
1277
894


,index,sample,Ne.1,Ne.2,Ne.3,Ne.4,Ne.5,Ne.6,Ne.7,Ne.8,...,Ne.117,Ne.118,Ne.119,Ne.120,Ne.121,Ne.122,Ne.123,Ne.124,Ne.125,Ne.126
0,383,19150000,-0.01974807131749937,-0.021139188192626515,-0.053390347201062636,-0.08355972019373291,-0.1126141101108204,-0.06468072961036891,0.08349598664594166,0.2525908879821148,...,1.0582063421205372,0.8081107307329922,0.562664480705231,0.31598843321836195,0.056545509579930056,-0.23298951024524042,-0.5055623266930582,-0.7762737422980249,-1.0662808906803078,-1.2875498218014267
1,384,19200000,-0.3958454079482879,-0.2128865437273673,-0.07103469567101413,-0.010770808612721313,0.020150763389631017,0.10706755770816813,0.20143458679649373,0.2663388572741314,...,1.054750096278383,0.8662868147518007,0.5882188862984119,0.31527765288448856,-0.04449833311530027,-0.38587432345563516,-0.5810823747066487,-0.7951710620533423,-0.935557152478665,-1.0519523959969388
2,385,19250000,-0.045126691747818935,-0.03152381606911732,-0.04637049314430266,-0.07927576027898098,-0.05504900794467146,-0.06253555958670391,-0.005464824694701284,0.07408889530603333,...,1.2279054434726546,0.9736828131491428,0.6903532784344621,0.3130781994767208,-0.12324753105839499,-0.4036225973870427,-0.6126766432929116,-0.7944057555197058,-0.9149292828378465,-0.9930743525912448
3,386,19300000,-0.1636377698092423,-0.0638354891326555,-0.060219273637605444,-0.053657876990129805,-0.0061196986247734906,0.06803597772594643,0.14880770908845364,0.27586007225310166,...,1.2050945695305004,0.9243264738040596,0.6137449978579463,0.2664721464745624,-0.10667866709369873,-0.4565772250447531,-0.7178926357530865,-0.9506530181999049,-1.130621238501644,-1.1185795800316094
4,387,19350000,-0.5866997790800755,-0.3045505978207238,-0.0990486181683034,-0.038684282297731534,-0.10998261550206875,-0.13704829359075674,-0.06498022235690598,0.13325103776653197,...,1.099871600912449,0.7843833090273195,0.48868959405118206,0.23714861793614844,-0.005431973927790566,-0.337708274602574,-0.5227769375334458,-0.7502746509807707,-0.9283824650679522,-1.2350017249645853


In [594]:
exp_ne_df = Ne_df.copy()
# Select only numeric columns excluding the first two columns
numeric_columns = exp_ne_df.columns[2:]  # Exclude the first two columns

# Apply the exponential function only to numeric columns
exp_ne_df[numeric_columns] = exp_ne_df[numeric_columns].apply(pd.to_numeric, errors='coerce').apply(np.exp)


uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

#calculate transmission rate based on SEIR dynamics
ne_diff_summary = generate_summary_diff_df(exp_ne_df)
seir_growth_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_growth_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.111,Ne.diff.112,Ne.diff.113,Ne.diff.114,Ne.diff.115,Ne.diff.116,Ne.diff.117,Ne.diff.118,Ne.diff.119,Ne.diff.120
0,78.643154,74.644665,70.010500,65.551888,62.013157,60.485296,61.889763,62.153244,62.514485,62.078238,...,110.045483,114.237375,115.786495,117.971573,121.973599,122.730515,122.783139,123.386292,124.694782,123.947924
1,67.319445,69.952281,70.045924,69.398127,69.366106,69.967616,67.921456,65.485690,66.245872,64.915880,...,110.977281,111.207455,113.816450,114.615737,118.816414,124.584315,124.897598,125.653607,121.617074,117.103312
2,80.158878,78.586945,76.502671,73.538145,70.352399,68.288184,65.601643,63.210425,62.064747,63.173491,...,106.986828,112.734661,115.706298,120.398311,127.827370,130.550833,130.998562,128.822675,123.999131,115.364119
3,73.749988,73.123223,70.990409,67.695619,65.364297,65.550412,67.399509,68.288871,69.002438,71.042578,...,107.143338,111.925306,117.883853,124.069344,129.301958,134.015893,133.492960,132.037051,128.114409,117.613103
4,68.999296,70.886983,70.243035,66.359266,61.890973,58.451331,58.752703,63.501091,69.433283,75.291786,...,109.679106,118.684576,123.350431,124.624569,125.864734,126.646433,124.509484,121.933837,118.531674,120.119610


In [595]:
ne_summary = generate_summary_df(Ne_df)
test = ne_summary
test['days'] = (test.interval.astype(int) -3 )*7
test['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - test.days.map(timedelta)
test = test.reset_index()

columns = ["mean_Ne_linear", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(6, min_periods =1).mean()



In [596]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.8, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50_MA',axis=alt.Axis(title="Effective Population Size", grid=False),scale=alt.Scale(domain=(0, 60))),
    alt.Y2('upper_hpd_linear_50_MA' ),
    
).properties(
    width=850,
    height=300)
# ).transform_filter(
#      (datum.upper_hpd_linear_50_MA < 50) 
#  )
band_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_MA',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_MA' ),
    
).properties(
    width=850,
    height=300
)

line_ne  + band_ne

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )


alt.LayerChart(...)

In [597]:
la_mpox_cases_df = pd.read_csv("../mpox_build/cluster_assignment/monkeypox_data.csv")
la_mpox_cases_df = la_mpox_cases_df.dropna(how = "all").dropna(axis = "columns", how = "all")
la_mpox_cases_df = la_mpox_cases_df.rename(columns= {"Unnamed: 0": "date"}); la_mpox_cases_df.head()

,date,cases
0,2022-05-19,1
1,2022-05-20,0
2,2022-05-21,0
3,2022-05-22,0
4,2022-05-23,0


In [598]:
cases = alt.Chart(la_mpox_cases_df).mark_bar(interpolate='monotone', opacity = 0.5, size = 1).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('cases',axis=alt.Axis(title="cases", grid=False))
).properties(
    width=850,
    height=300
)

(cases + line_ne).resolve_scale(y = "independent")

alt.LayerChart(...)

In [599]:
#switching to migration

In [600]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [601]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "immigrationRate." in i:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
#             try:
#                 next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_mig_log":mean_log,"mean_mig_linear":mean_linear, 
                                                   "median_mig_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
   
            

            
    return(new_df)

In [602]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

1277
894


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.116,immigrationRate.117,immigrationRate.118,immigrationRate.119,immigrationRate.120,immigrationRate.121,immigrationRate.122,immigrationRate.123,immigrationRate.124,immigrationRate.125
0,383,19150000,-0.2855252506561548,-1.0051133445426652,-0.7655451867018283,-1.17950313572161,-0.9009909953090672,-0.6447143843824952,-1.015292686416621,-0.630707720101144,...,1.6366276313062305,2.4595325343030416,2.9715708148164124,2.272770876475615,2.662473678042776,2.908016568474976,3.1094121505375427,3.2280246210890877,3.550221255369882,4.202394368569496
1,384,19200000,-1.3970925797791878,-0.9745178884575988,-1.9244444351169057,-1.2172098639668432,-0.9549660947927806,-0.7758298683215832,-0.7905670252507049,-0.38899487107181197,...,1.5140053227437746,2.289079868719371,2.83725668423525,2.38932645075601,3.0646122148894164,2.9385522069560883,3.4188411102496454,3.026040403758107,3.419611686864549,4.164797090462879
2,385,19250000,-0.7301562853998126,-0.9906810216185553,-1.8551161722138514,-1.0221489264417596,-1.7876628392388514,-0.9582238227045339,-0.8430741318454757,-1.4974911417637304,...,1.4730189543235939,2.128131012352149,2.3725377961063256,2.5097326432618234,3.0579612878889213,2.943227951141017,3.4143976741267417,2.987993799844724,3.4636575289619556,3.986756242345952
3,386,19300000,-1.1470216528274613,-1.6561819580555481,-1.7398406828227753,-1.1151211838197672,-1.8046544586944715,-1.2919826219098387,-1.5851271646726253,-1.3044415597961996,...,1.437599376529745,2.186241293906694,2.5089637216770955,2.8661981213728405,3.1350197110989355,3.082614132164168,3.5522619882992124,3.2680662359967076,3.3595824568714963,3.9521381974067253
4,387,19350000,-2.0961361409810393,-1.9009207456975117,-1.7751097187301803,-1.0209067256643545,-1.5614926385799865,-1.1115079363667926,-1.3267282557755373,-1.3503113415378858,...,1.7491553422278894,2.0663784168617454,2.6633677510897185,2.8337753661556366,3.3203747398289143,3.9039757696054553,3.806602813420007,3.4328976451876687,3.509647800561631,3.5123905608138326


In [603]:
mig_summary = generate_summary_df(mig_df_f)
mig_summary.head()

,interval,mean_mig_log,mean_mig_linear,median_mig_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear,upper_hpd_linear_50,lower_hpd_linear_50
0,1,-0.733635,0.480160,-0.714480,0.993553,-2.720161,-0.142532,-1.344362,2.700813,0.065864,0.867160,0.260706
0,2,-0.737931,0.478102,-0.698826,1.047464,-2.510779,-0.104142,-1.173152,2.850414,0.081205,0.901097,0.309390
0,3,-0.745694,0.474405,-0.694477,0.754366,-2.442231,-0.001867,-1.008188,2.126264,0.086967,0.998135,0.364880
0,4,-0.713675,0.489841,-0.670468,0.643774,-2.299354,-0.126109,-1.069449,1.903653,0.100324,0.881519,0.343197
0,5,-0.660668,0.516506,-0.619129,0.649387,-1.998604,-0.103305,-1.056036,1.914366,0.135524,0.901852,0.347832


In [604]:
#format dates
test_mig = mig_summary
test_mig['days'] = (test_mig.interval.astype(int) -3 )*7
test_mig['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - test_mig.days.map(timedelta)

In [605]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.8).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    
).properties(
    width=850,
    height=300
)

band_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False), scale=alt.Scale(domain=(0, 100))),
    alt.Y2('upper_hpd_linear' ),
    
).properties(
    width=850,
    height=300
)


mig_plot = (line_mig + band_mig)
mig_plot

alt.LayerChart(...)

In [606]:
## percentage of cases due to introductions

In [607]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_growth_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [608]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [609]:
percent_df = generate_percent_intro_df(mig_df_f)
percent_df.head()

,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.111,intro.percent.112,intro.percent.113,intro.percent.114,intro.percent.115,intro.percent.116,intro.percent.117,intro.percent.118,intro.percent.119,intro.percent.120
0,0.009467,0.004879,0.006599,0.004668,0.006507,0.008602,0.005820,0.008490,0.008061,0.021516,...,0.037725,0.042566,0.063649,0.102555,0.093273,0.040181,0.086995,0.136608,0.072219,0.103643
1,0.003660,0.005366,0.002079,0.004248,0.005517,0.006536,0.006634,0.010243,0.007441,0.032728,...,0.045742,0.045753,0.077640,0.122533,0.085829,0.035197,0.073209,0.119595,0.082296,0.154669
2,0.005975,0.004703,0.002041,0.004869,0.002373,0.005586,0.006518,0.003526,0.005865,0.015796,...,0.037139,0.042409,0.087387,0.110335,0.070998,0.032335,0.060253,0.076853,0.090254,0.155758
3,0.004288,0.002603,0.002467,0.004820,0.002511,0.004174,0.003031,0.003957,0.005079,0.011569,...,0.036240,0.046837,0.089872,0.092888,0.069872,0.030461,0.062514,0.085168,0.120604,0.163504
4,0.001778,0.002104,0.002407,0.005400,0.003379,0.005598,0.004496,0.004065,0.004344,0.011589,...,0.036915,0.028541,0.070120,0.098231,0.062442,0.043428,0.059636,0.105259,0.125494,0.187230


In [610]:
summary_percent_df = generate_summary_df(percent_df)
summary_percent_df = summary_percent_df.reset_index()

#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 1.5 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date
0,0,1,0.012484,0.038343,0.000179,0.007564,0.000806,-14,2024-09-26
1,0,2,0.011639,0.034860,0.000414,0.007573,0.000801,-7,2024-09-19
2,0,3,0.010806,0.031594,0.000380,0.009553,0.002785,0,2024-09-12
3,0,4,0.010696,0.027262,0.000744,0.009011,0.002612,7,2024-09-05
4,0,5,0.011053,0.029051,0.001041,0.009237,0.002105,14,2024-08-29


In [611]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(4, min_periods =1).mean()


In [612]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .7))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=850,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .7))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=850,
    height=300
)


intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.LayerChart(...)

In [613]:
## calculate Rt

In [614]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [615]:
def generate_local_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            try:
                new_df["rt" + "." + str(interval)] = ((1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))) *  (1- percent_df["intro.percent."+str(interval)].astype("float"))
                
            except KeyError:
                pass

            
    return(new_df)

In [616]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [617]:
# a line to show an Rt of 1
one_line = alt.Chart(pd.DataFrame({'y': [1.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

In [618]:
#read in data and calculate based on posterior set of trees
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
rt_local_df = generate_local_rt(ne_diff_summary)

#summarize data
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_df = generate_rt_summary_df(rt_local_df)

summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50
0,1,0.846786,0.838965,1.398321,0.310131,1.028628,0.682247
0,2,0.825399,0.824673,1.339161,0.372092,0.978514,0.674982
0,3,0.799212,0.800525,1.242517,0.435940,0.957429,0.700763
0,4,0.776702,0.778092,1.137522,0.451778,0.886624,0.659009
0,5,0.760875,0.768720,1.023360,0.466442,0.887385,0.694073


In [619]:
summary_rt_local_and_intro_df['days'] = (summary_rt_local_and_intro_df.interval.astype(int) -3) *7
summary_rt_local_and_intro_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_rt_local_and_intro_df.days.map(timedelta)

summary_rt_local_df['days'] = (summary_rt_local_df.interval.astype(int) -3) *7
summary_rt_local_df['date'] = dt.strptime("2024-09-12",  "%Y-%m-%d") - summary_rt_local_df.days.map(timedelta)

summary_rt_local_and_intro_df["contribution"] = "Local + Introductions"
summary_rt_local_df["contribution"] = "Local"

In [620]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(6, min_periods =1).mean()
    summary_rt_local_df[f'{column}_MA'] = summary_rt_local_df[column].rolling(6, min_periods =1).mean()


In [621]:
combined_rt_df = pd.concat([summary_rt_local_and_intro_df,summary_rt_local_df], ignore_index = True )

In [622]:
line2 = alt.Chart(combined_rt_df).mark_area(interpolate='monotone', opacity = 1, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50_MA',axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Y2('upper_hpd_log_50_MA' ),
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)
band2 = alt.Chart(combined_rt_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_95_MA', axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95_MA'), 
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)

line2 + band2 + one_line

alt.LayerChart(...)

In [623]:
line_simple_intro = alt.Chart(combined_rt_df).mark_line(interpolate='monotone', opacity = 1, clip = True, strokeDash=[5,1]).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('mean_percent_MA',scale=alt.Scale(domain=[0.5, 2]),axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Color("contribution", legend=alt.Legend(offset=-150))

).properties(
    width=850,
    height=300
)


line_rt = line_simple_intro+ one_line; line_rt

alt.LayerChart(...)

In [624]:
alt.layer(intro_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [625]:
alt.layer(mig_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [626]:
cases_ne = (cases + line_ne).resolve_scale(y= "shared")

In [627]:
alt.layer(cases_ne, line_rt).resolve_scale(y = "independent")

alt.LayerChart(...)

In [628]:
## compare phylo rt to microsim Rt

In [767]:
sim_df = pd.read_csv("/Users/mparedes/Downloads/results_sep13_2024.csv")
sim_df["week"] = sim_df.date.str.rsplit(" ", 0).str[-0].tolist()
sim_df = sim_df.dropna()

sim_df["Model (FoI=1) R_e"]= sim_df["Model (FoI=1) R_e"].astype("float")
sim_df["Model (FoI=1.25) R_e"]= sim_df["Model (FoI=1.25) R_e"].astype("float")
sim_df["Model (FoI=1.5) R_e"]= sim_df["Model (FoI=1.5) R_e"].astype("float")

sim_df.head()

,date,Model (FoI=1) R_e,Model (FoI=1.25) R_e,Model (FoI=1.5) R_e,week
0,7/10/22 - 7/16/22,2.320442,2.320442,2.320442,7/10/22
1,7/17/22 - 7/23/22,2.677419,2.677419,2.677419,7/17/22
2,7/24/22 - 7/30/22,2.311224,2.311224,2.311224,7/24/22
3,7/31/22 - 8/6/22,2.369697,2.369697,2.369697,7/31/22
4,8/7/22 - 8/13/22,0.693053,0.693053,0.693053,8/7/22


In [783]:
columns = ["Model (FoI=1) R_e","Model (FoI=1.25) R_e", "Model (FoI=1.5) R_e"]
for column in columns:
    # Create a new column for the moving average
    sim_df[f'{column}_MA'] = sim_df[column].rolling(3, min_periods =1, center = True).mean()



In [784]:
sim_df.head()

,date,Model (FoI=1) R_e,Model (FoI=1.25) R_e,Model (FoI=1.5) R_e,week,Model (FoI=1) R_e_MA,Model (FoI=1.25) R_e_MA,Model (FoI=1.5) R_e_MA
0,7/10/22 - 7/16/22,2.320442,2.320442,2.320442,7/10/22,2.498931,2.498931,2.498931
1,7/17/22 - 7/23/22,2.677419,2.677419,2.677419,7/17/22,2.436362,2.436362,2.436362
2,7/24/22 - 7/30/22,2.311224,2.311224,2.311224,7/24/22,2.452780,2.452780,2.452780
3,7/31/22 - 8/6/22,2.369697,2.369697,2.369697,7/31/22,1.791325,1.791325,1.791325
4,8/7/22 - 8/13/22,0.693053,0.693053,0.693053,8/7/22,1.245723,1.245723,1.245723


In [785]:
long_sims_df = pd.melt(sim_df, id_vars='week', value_vars=['Model (FoI=1) R_e_MA', 'Model (FoI=1.25) R_e_MA', 'Model (FoI=1.5) R_e_MA'])
long_sims_df.week = pd.to_datetime(long_sims_df.week)



In [786]:
subset_sims_df = long_sims_df[long_sims_df.week >"2023-03-01"]
#subset_sims_df.variable[subset_sims_df.variable == "Empirical # of diagnosed cases_MA"] = "Empirical"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1) R_e_MA"] = "Model (FoI = 1)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.25) R_e_MA"] = "Model (FoI = 1.25)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.5) R_e_MA"] = "Model (FoI = 1.5)"



In [787]:
subset_sims_df

,week,variable,value
34,2023-03-05,Model (FoI = 1),0.250000
35,2023-03-12,Model (FoI = 1),0.000000
36,2023-03-19,Model (FoI = 1),0.000000
37,2023-03-26,Model (FoI = 1),0.000000
38,2023-04-02,Model (FoI = 1),0.000000
...,...,...,...
328,2024-07-21,Model (FoI = 1.5),1.022222
329,2024-07-28,Model (FoI = 1.5),1.293651
330,2024-08-04,Model (FoI = 1.5),1.365079
331,2024-08-11,Model (FoI = 1.5),1.920635


In [795]:
stream_graph_all =  alt.Chart(long_sims_df, title = "").mark_line(interpolate='monotone', opacity = 1 ,color = "#f58518", clip = True).encode(
    alt.X('week:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('value',axis=alt.Axis(title="Estimated Rt", grid=False, labelFontSize = 16),stack = False),

    alt.Color('variable:N',legend=alt.Legend(title= "Estimated Rt",offset = -100, labelFontSize = 10, titleFontSize = 10)),


).properties(
    width=800,
    height=300
)#.transform_filter(datum.variable != "Empirical")

In [796]:
stream_graph_all 

alt.Chart(...)

In [792]:
case_rt = pd.read_csv("../scripts/case-rt-estimates_region.tsv", sep = "\t"); case_rt.head()

FileNotFoundError: [Errno 2] No such file or directory: '../scripts/case-rt-estimates_region.tsv'

In [635]:
case_rt_plot = alt.Chart(case_rt, title = "").mark_line(interpolate='monotone', opacity = 0.8 ,color = "black", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('median_R',axis=alt.Axis(title="Estimated Rt", grid=False),stack = False),



).properties(
    width=800,
    height=300
)
case_rt_plot

alt.Chart(...)

In [797]:
stream_graph_all + line_rt 

alt.LayerChart(...)